# Plotting functions to accompany plot_explore_GCB.ipynb

Humans and the Carbon Cycle, Fall 2024 (EESC GU 4020) at Columbia University \
August 7 2024 draft1 1, A. Shaum and Professor G.A. McKinley \
September 7 2024 for Problem Set 1

In [ ]:
pip install openpyxl

In [ ]:
import pandas as pd
import openpyxl # <- needed to open xlsx
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget 
# ^ allows zooming in to the plots!

import cmocean as cm    

import gcsfs
fs = gcsfs.GCSFileSystem()

In [ ]:
def GCB_plot(time, total_emissions,land_use,ocean_sink,land_sink,oceanfactor=1,landfactor=1):    
    """
    Recreates Global Carbon Budget figure 3. 

    Args:
        time(pd.series): Time range from dataframe 
        total_emissions(pd.series): Fossil fuel emissions + land-use emissions
        land_use(pd.series): Land-use emissions
        ocean_sink(pd.series): Ocean-carbon sink data
        land_sink(pd.series): Land-carbon sink data
        oceanfactor(float): Strength of ocean sink (functions as a multiplier)
        landfactor(float): Strength of land sink (functions as a multiplier)
    """
    ocean = oceanfactor*ocean_sink
    ocean_land = ocean + landfactor*land_sink
    ocean_land_atmo = ocean_land + atmo_sink + (1-oceanfactor)*ocean_sink + (1-landfactor)*land_sink
    
    # the sink-equivalent of total emissions: where the total sink would need to be to achieve net-0 Carbon emissions
    mirror_total_emissions = -total_emissions

    fig = plt.figure(figsize=(10,6))
    if oceanfactor != 1 or landfactor != 1:
        plt.title(f'Global Carbon Budget (as GCB2023 Fig 3). Scalings: ocean = {oceanfactor}, land = {landfactor}')
    elif oceanfactor == 1 and landfactor == 1:
        plt.title(f'Global Carbon Budget (as GCB2023 Fig 3).')

    plt.ylabel('GtC/Yr')
    plt.xlabel('Year')

    # plotting curves
    plt.plot(time,total_emissions,'#e25f4a',label = "TOTAL emissions: FF (gray) + LandUse (orange)",linewidth=2)
    plt.plot(time,land_use,'#efac39',linewidth=2)
    plt.plot(time,ocean, '#3ca490',label = "GCB ocean",linewidth=2)
    plt.plot(time,ocean_land,'#72be4c',label = "GCB ocean + land",linewidth=2)
    plt.plot(time,ocean_land_atmo,'#0ebcf6',label = "GCB ocean+land+atmosphere",linewidth=2)
    plt.plot(time,mirror_total_emissions,'#e25f4a',linestyle='dashed',label = "mirror total emissions",linewidth=1)

    # adding fill
    plt.fill_between(time, 0, ocean_land_atmo, facecolor='#32c7f8')
    plt.fill_between(time, 0, ocean_land, facecolor='#95c959')
    plt.fill_between(time, 0, ocean, facecolor='#5eb5a4')
    plt.fill_between(time, 0, total_emissions, facecolor='#879fab')
    plt.fill_between(time,0,land_use,facecolor='#f5bd5c')

    # miscellaneous 
    plt.grid(True)
    plt.legend(frameon = True)
    plt.xlim(1850,2022)
    plt.axhline(0,c='black')

    return fig

In [ ]:
def GCBcompare_scale(time, total_emissions,land_use,ocean_sink,land_sink,oceanfactor=1,landfactor=1): 
    """
    Compares original Global Carbon Budget figure 3 to version with scaling 

    Args:
        time(pd.series): Time range from dataframe 
        total_emissions(pd.series): Fossil fuel emissions + land-use emissions
        land_use(pd.series): Land-use emissions
        ocean_sink(pd.series): Ocean-carbon sink data
        land_sink(pd.series): Land-carbon sink data
        oceanfactor(float): Strength of ocean sink (functions as a multiplier)
        landfactor(float): Strength of land sink (functions as a multiplier)
    """

    # Plot emission
    fig = plt.figure(figsize=(10,6))
    plt.plot(time,total_emissions,'#e25f4a',label = "TOTAL emissions: FF + LandUse",linewidth=3)
    plt.plot(time,land_use,'#efac39',linewidth=3)
    
    # calculate and plot scaled fluxes
    ocean = oceanfactor*ocean_sink
    ocean_land = ocean + landfactor*land_sink
    ocean_land_atmo = ocean_land + atmo_sink + (1-oceanfactor)*ocean_sink + (1-landfactor)*land_sink
 
    plt.plot(time,ocean,'#3ca490',label = f"GCB ocean, scaling: ocean = {oceanfactor}",linewidth=3)
    plt.plot(time,ocean_land,'#72be4c',label = f"GCB ocean + land, scaling: land = {landfactor}",linewidth=3)
    plt.plot(time,ocean_land_atmo,'#0ebcf6',label = "GCB ocean+land+atmosphere",linewidth=3)

    # Rest factors and replot 
    oceanfactor=1
    landfactor=1
    ocean = oceanfactor*ocean_sink
    ocean_land = ocean + landfactor*land_sink
    ocean_land_atmo = ocean_land + atmo_sink + (1-oceanfactor)*ocean_sink + (1-landfactor)*land_sink

    plt.plot(time,ocean_sink,'#3ca490',linewidth=1, label='GCB ocean, scaling: ocean = 1')
    plt.plot(time,ocean_land,'#72be4c',linewidth=1,label='GCB ocean + land, scaling: land = 1')
    plt.plot(time,ocean_land_atmo,'#0ebcf6',linewidth=1)

    # plot mirrored emissions
    mirror_total_emissions = -total_emissions
    plt.plot(time,mirror_total_emissions,'black',linestyle='dashed',label = "mirror total emissions",linewidth=1)

    # finish plot
    plt.grid(True)
    plt.legend(frameon = True)
    plt.ylabel('GtC/yr')
    plt.xlabel('Year')
    plt.axhline(0,c='black')
    plt.xlim(1850,2022)
    plt.title('Global Carbon Budget 2023, comparison with and without scaling.')
    # plt.ylim(-12,0)

In [ ]:
def bar_plot(fossil,land_use,land_sink,ocean_sink,atm_growth,oceanfactor=1, landfactor=1):
    """
    Bar plot of carbon emissions and carbon sinks. 

    fossil(pd.series): Fossil fuel emissions
    land_use(pd.series): Land-use emissions
    land_sink(pd.series): Land-carbon sink data
    ocean_sink(pd.series): Ocean-carbon sink data
    atm_growth(pd.series): Atmospheric-carbon uptake data
    oceanfactor(float): Strength of ocean sink (functions as a multiplier)
    """

    cumulative = np.zeros(5)
    cumulative[0] = np.sum(fossil)
    cumulative[1] = np.sum(land_use)
    cumulative[2] = np.sum(landfactor*land_sink)
    cumulative[3] = np.sum(oceanfactor*ocean_sink)
    cumulative[4] = np.sum(atm_growth+(1-oceanfactor)*ocean_sink+(1-landfactor)*land_sink)
    
    barlabel = ['Fossil Fuel','Land Use','Land','Ocean','Atmosphere']
    
    fig = plt.figure(figsize=(10,6))
    plt.ylim((-500,500))
    plt.bar(barlabel,cumulative,color='thistle',label='normal ocean sink')
    plt.ylabel('GtC')
    if oceanfactor != 1 or landfactor != 1:
        plt.title(f'CO$_2$ Sources (positive) and Sinks (negative), 1850-2022, scaling: ocean={oceanfactor}, land={landfactor}')
    elif oceanfactor == 1 and landfactor == 1:
        plt.title(f'CO$_2$ Sources (positive) and Sinks (negative), 1850-2022')
    plt.axhline(0,color='black')

In [ ]:
def atmospheric_co2_plot(GCB23historical1959, xCO2atm_ML, oceanfactor=1, landfactor=1, atmCO2_1958=315):
    """
    Plots Global Carbon Budget historical atmospheric CO2 levels compared to the monthly observed CO2 values at Mauna Loa.

    Args:
        GCB23historical1959(pd.Dataframe): Dataframe of Global Carbon Budget data. In this function, we use 'atmospheric growth' data,
            convert it to PPM (parts per million), and add it to the initial CO2 value from 1958. The 'atmospheric growth' measures
            growth since 1958, hence the need to add the initial value.
            
        xCO2atm_ML(pd.Dataframe): Dataframe of atmospheric CO2 levels observed at Mauna Loa, monthly. 
        
        oceanfactor(float): Multiplier to increase or decrease strength of ocean carbon sink. 
            Defaults to 1, means based on real ocean carbon sink.
            
        atmCO2_1958(float): Initial atmospheric CO2 levels, which gets added on top of the atmospheric CO2 growth data from the 
            Global Carbon Budget. Value in 1958 was 315ppm, so this is the default

    """
    convertGtC_to_ppm = 2.124 # Factor to convert from GtC in atmosphere to ppm,
    #Global Carbon Budget 2023 (Friedlingstein et al. 2023) " 1ppm=2.124GtC "
    
    atmCO2 =np.cumsum((GCB23historical1959['atmospheric growth']+(1-oceanfactor)*GCB23historical1959['ocean sink']+(1-landfactor)*GCB23historical1959['land sink']))/convertGtC_to_ppm+atmCO2_1958
    #adds 1959 co2 onwards to the initial/starting value in 1958
    
    fig = plt.figure(figsize=(10,6))
    plt.title(f'GCB Atmospheric CO2 vs CO2 Observations at Mauna Loa, scaling: ocean={oceanfactor}, land={landfactor}')
    plt.ylabel('co2 (ppm)')
    plt.xlabel('time (years)')

    if oceanfactor != 1 or landfactor != 1:
        plt.plot(GCB23historical1959['Year']+0.5,atmCO2,'cyan',marker ='x',label = "Modified integration of GCB atmospheric growth, annual",linewidth=3,markersize=7)
    elif oceanfactor == 1 and landfactor == 1:
        plt.plot(GCB23historical1959['Year']+0.5,atmCO2,'cyan',marker ='x',label = "GCB atmospheric growth, integrated and converted to ppm, annual",linewidth=3,markersize=7)
    
    plt.plot(xCO2atm_ML['year']+(xCO2atm_ML['month']-0.5)/12,xCO2atm_ML['average'],'red',marker ='.',label = "CO$_2$ observations at Mauna Loa, monthly",linewidth=1,markersize=0.8)

    # misc plotting
    plt.grid(True)
    plt.legend(frameon = True)
